In [ ]:
import os
import numpy as np
import nibabel as nib
from tqdm import tqdm
from multiprocessing import Pool

# Configuration
data_dir = '/content/drive/MyDrive/Processed'
output_dir = '/content/drive/MyDrive/Extracted_ROIs'
class_folders = ['CN_Combined', 'MCI_Combined', 'AD_Combined']

# Neuropathology Regions (DKT Atlas labels)
regions = {
    'Aβ': {
        'Precuneus': 1023,
        'Posterior_Cingulate': 1024,
        'Frontal_Cortex': [1003, 1012]  # Superior and Inferior Frontal
    },
    'NFT': {
        'Entorhinal': 1006,
        'CA1': 1026,
        'Amygdala': 18
    }
}

def create_mask(seg_data, labels):
    mask = np.zeros_like(seg_data, dtype=bool)
    if isinstance(labels, list):
        for label in labels:
            mask |= (seg_data == label)
    else:
        mask = (seg_data == labels)
    return mask

def process_subject(subject_path):
    try:
        # Load FastSurfer outputs
        seg_file = os.path.join(subject_path, 'aparc.DKTatlas+aseg.deep.mgz')
        orig_file = os.path.join(subject_path, 'orig.mgz')

        seg_img = nib.load(seg_file)
        orig_img = nib.load(orig_file)
        seg_data = seg_img.get_fdata()
        orig_data = orig_img.get_fdata()

        # Create output directory
        relative_path = os.path.relpath(subject_path, data_dir)
        output_path = os.path.join(output_dir, relative_path)
        os.makedirs(output_path, exist_ok=True)

        # Extract pathology-specific ROIs
        for pathology in ['Aβ', 'NFT']:
            combined_mask = np.zeros_like(seg_data, dtype=bool)
            for region, labels in regions[pathology].items():
                combined_mask |= create_mask(seg_data, labels)

            # Apply mask and save
            roi_data = orig_data * combined_mask
            output_file = os.path.join(output_path, f'{pathology}_ROI.nii.gz')
            nib.save(nib.Nifti1Image(roi_data, orig_img.affine), output_file)

    except Exception as e:
        print(f"Error processing {subject_path}: {str(e)}")

def main():
    # Collect all subject paths
    subjects = []
    for cls in class_folders:
        class_dir = os.path.join(data_dir, cls)
        for subject in os.listdir(class_dir):
            subject_dir = os.path.join(class_dir, subject, 'mri')
            if os.path.exists(os.path.join(subject_dir, 'aparc.DKTatlas+aseg.deep.mgz')):
                subjects.append(subject_dir)

    # Process with parallel workers (set to 4 for Colab CPU)
    with Pool(processes=4) as pool:
        list(tqdm(pool.imap(process_subject, subjects), total=len(subjects)))

if __name__ == '__main__':
    # Mount Google Drive first
    from google.colab import drive
    drive.mount('/content/drive')

    # Create output directory
    os.makedirs(output_dir, exist_ok=True)

    # Run main processing
    main()
    print("Processing completed. Extracted ROIs saved in:", output_dir)

Mounted at /content/drive


100%|██████████| 2180/2180 [58:39<00:00,  1.61s/it]

Processing completed. Extracted ROIs saved in: /content/drive/MyDrive/Extracted_ROIs
